# AC109 Project Modeling Results: Predicting the returns on Cryptocurrencies

by Ali Dastjerdi, Angelina Massa, Sachin Mathur & Nate Stein

### Supporting Libraries

We outsourced some of the supporting code to other modules we wrote located in the main directory with the intent of having this notebook focus on the presentation of results. The supporting modules are:
- `crypto_utils.py` contains the code we used to scrape and clean data from coinmarket.cap. It also contains the code used to wrangle/preprocess that data (saved in CSV files) into our design matrix. By separating the creation of the design matrix in its own `.py` file, we were also able to create unit tests to ensure the resulting figures match what we expected based on hand-calculated figures, which became increasingly important as we engineered more involved features.
- `crypto_models.py` contains the code we used to iterate over multiple classification and regression models and summarize the results in tabular form.

In [40]:
import crypto_utils as cryp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics
import time as time

from crypto_utils import fmt_date, print_update

In [2]:
# Custom output options.

np.set_printoptions(precision=4, suppress=True)
pd.set_option('display.precision', 4)
sns.set_style('white')
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 14
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['savefig.bbox'] = 'tight'
plt.rcParams['savefig.pad_inches'] = 0.05
%matplotlib inline

## Construct Design Matrix

In [3]:
# We will not be using any non-crypto assets at the moment.

x_cryptos = ['ltc', 'xrp', 'xlm', 'eth']
y_crypto = 'btc'
kwargs = {'n_rolling_price':1, 'n_rolling_volume':2,
          'x_assets':[], 'n_std_window':30}
design = cryp.DesignMatrix(x_cryptos=x_cryptos, y_crypto=y_crypto, **kwargs)
X, Y = design.get_data()
print('Dimensions:')
print('X: {}'.format(X.shape))
print('Y: {}'.format(Y.shape))

Dimensions:
X: (949, 10)
Y: (949,)


In [6]:
# Seperate into train/test.

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, Y, test_size=0.2, random_state=60)

# Modeling

In [23]:
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

### Baseline Model

In [19]:
lr = LinearRegression().fit(X_train, y_train)
mae_lr = metrics.mean_absolute_error(y_test, lr.predict(X_test))
print('Baseline LinearRegression model MAE: {:.2%}'.format(mae_lr))

Baseline LinearRegression model MAE: 3.00%


### XGBRegressor

In [34]:
def build_xgb_model(X_train, y_train):
    """Iterate over a hyperparameter space and return best model on a 
    validation set reserved from input training data.
    """
    # Define hyperparam space.
    expon_distr = stats.expon(0, 50)
    cv_params = {
        'n_estimators': stats.randint(4, 100),
        'max_depth': stats.randint(2, 100),
        'learning_rate': stats.uniform(0.05, 0.95),
        'gamma': stats.uniform(0, 10),
        'reg_alpha': expon_distr,
        'min_child_weight': expon_distr
    }

    # Iterate over hyperparam space.
    xgb = XGBRegressor(nthreads=-1)  # nthreads=-1 => use max cores
    
    print_update('Tuning XGBRegressor hyperparams...')
    t0 = time.time()
    gs = RandomizedSearchCV(xgb, cv_params, n_iter=400, n_jobs=1, cv=4, 
                            random_state=88)
    gs.fit(X_train, y_train)
    print_update('Finished tuning XGBRegressor in {:.0f} secs.'.format(
        time.time() - t0))
    
    return gs.best_estimator_

In [35]:
xgb = build_xgb_model(X_train, y_train)

In [37]:
mae_xgb = metrics.mean_absolute_error(y_test, xgb.predict(X_test))
print('XGBRegressor MAE: {:.2%}'.format(mae_xgb))

XGBRegressor MAE: 2.98%
